In [1]:
import pandas as pd
import numpy as np


reviews = pd.read_csv('C:/Users/lenovo/Downloads/review_data.csv')
reviews = reviews.drop(columns=['review_id', 'bank_id', 'user_id', 'date_posted'], axis=0)
reviews.head()

FileNotFoundError: [Errno 2] File b'C:/Users/lenovo/Downloads/review_data.csv' does not exist: b'C:/Users/lenovo/Downloads/review_data.csv'

In [3]:
reviews.groupby('bank_name', as_index=False).review.count()

,bank_name,review
0,Ally Bank,703
1,American Express National Bank,261
2,Associated Credit Union,5
3,Bank of America,343
4,CIT Bank,296
5,Capital One,411
6,Chase Bank,220
7,Citi,129
8,Delta Community Credit Union,26
9,Georgia Banking Company,8


In [4]:
final_reviews = reviews[reviews['bank_name']=='Ally Bank']
final_reviews.reset_index(inplace=True)
final_reviews = final_reviews.drop(columns=['index'], axis=0)
# final_reviews.loc[378,:]

In [5]:
#change any int/float to str
final_reviews['review']= final_reviews['review'].apply(str)

# Convert the reviews to lowercase
final_reviews['review']= final_reviews['review'].map(lambda x: x.lower())

In [6]:
# !pip install -U gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [6]:
# from gensim.parsing.preprocessing import strip_punctuation
# strip_punctuation(final_reviews['review'])

# # from gensim.parsing.preprocessing import remove_stopwords

# # remove_stopwords(final_reviews['review'])


In [7]:
# #lemmatize — words in third person are changed to first person and verbs in past and future tenses are changed into present
# from nltk import PorterStemmer
# stemmer = PorterStemmer()
# def lemmatize_stemming(text):
#     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# #Stemming - words are reduced to their root form
# def preprocess(text):
#     result = []
#     for token in gensim.utils.simple_preprocess(text):
#         if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
#     return result

In [19]:
def preprocess(text):
   # split into words
    text = word_tokenize(text)
    # convert to lower case
    text= [w.lower() for w in text]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    text = [w.translate(table) for w in text]
    # remove remaining tokens that are not alphabetic
    text = [word for word in text if word.isalpha()]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # ....
#     # stemming of words
#     porter = PorterStemmer()
#     text = [porter.stem(word) for word in text]
    
    # join all
    text = " ".join(text)
    return(text)

In [20]:
#Checking with sample review
doc_sample = final_reviews.loc[500,'review']
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['i', 'opened', 'a', 'checking', 'account', 'with', 'ally', 'after', 'my', 'regular', 'walk-in', 'bank', 'started', 'bsing', 'me', 'around', 'with', 'fees.', 'the', 'fees', 'at', 'ally', 'are', 'reasonable', 'and', 'fair.', 'customer', 'service', 'has', 'always', 'been', 'there', 'for', 'me,', 'regardless', 'of', 'the', 'problem.']


 tokenized and lemmatized document: 
opened checking account ally regular walkin bank started bsing around fees fees ally reasonable fair customer service always regardless problem


In [25]:
#Apply to all reviews
final_reviews['reviews_processed'] = pd.DataFrame(final_reviews['review'].map(preprocess))
final_reviews.head(5)
type(final_reviews['reviews_processed'])
(final_reviews['reviews_processed']).str.split()

0      [advised, ally, bank, accept, money, orders, i...
1      [zelle, payment, payments, placed, hold, requi...
2      [every, single, transaction, attempted, ally, ...
3      [deal, ira, department, moved, ira, sep, ally,...
4      [ally, plus, years, least, different, cd, reg,...
                             ...                        
698    [setting, acct, ally, initiated, fund, transfe...
699    [ally, bank, money, market, checkingonline, sa...
700    [good, thing, ally, bank, cd, day, penalty, wi...
701    [agree, totally, assessment, ally, extremely, ...
702    [great, bank, fast, ach, offered, loyalty, bum...
Name: reviews_processed, Length: 703, dtype: object

In [ ]:
#Word cloud -- visual representation of most common words
#!pip install wordcloud
from wordcloud import WordCloud
long_string = ','.join(list(final_reviews['review'].values))
wordcloud = WordCloud(background_color="black", max_words=5000, contour_width=3, contour_color='steelblue')
wordcloud.generate(long_string)
wordcloud.to_image()

# Bag of Words

In [36]:

dictionary = gensim.corpora.Dictionary((final_reviews['reviews_processed']).str.split())
del_ids = [k for k,v in dictionary.items() if (v=='nt' or v=='br')]
dictionary.filter_tokens(bad_ids=del_ids)
# count = 0
# for k, v in dictionary.iteritems():
#     print(k, v)
#     count += 1
#     if count > 10:
#         break

In [37]:
#Filter out tokens that appear in
    #less than 15 documents (absolute number) or
    #more than 0.5 documents (fraction of total corpus size, not absolute number).
    #after the above two steps, keep only the first 100000 most frequent tokens.

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [38]:
bow_corpus = [dictionary.doc2bow(doc) for doc in (final_reviews['reviews_processed'].str.split())]
bow_corpus[500]

[(59, 1),
 (72, 2),
 (118, 1),
 (130, 1),
 (182, 1),
 (342, 1),
 (358, 1),
 (373, 1),
 (386, 1)]

# Running LDA model

In [39]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=20, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"money" + 0.025*"check" + 0.020*"told" + 0.018*"days" + 0.018*"back" + 0.016*"transfer" + 0.016*"deposit" + 0.015*"get" + 0.015*"call" + 0.014*"called"
Topic: 1 
Words: 0.053*"fees" + 0.030*"atm" + 0.024*"fee" + 0.022*"overdraft" + 0.018*"checking" + 0.016*"online" + 0.015*"money" + 0.014*"card" + 0.014*"deposit" + 0.014*"love"
Topic: 2 
Words: 0.037*"savings" + 0.036*"interest" + 0.027*"money" + 0.018*"also" + 0.018*"rates" + 0.017*"online" + 0.017*"fees" + 0.016*"accounts" + 0.016*"rate" + 0.016*"checking"
Topic: 3 
Words: 0.025*"money" + 0.022*"rates" + 0.021*"accounts" + 0.017*"always" + 0.017*"cd" + 0.017*"would" + 0.016*"one" + 0.015*"time" + 0.015*"opened" + 0.014*"banks"
Topic: 4 
Words: 0.030*"app" + 0.030*"easy" + 0.024*"use" + 0.020*"mobile" + 0.014*"money" + 0.014*"also" + 0.014*"online" + 0.014*"banking" + 0.014*"phone" + 0.014*"using"
Topic: 5 
Words: 0.024*"credit" + 0.022*"time" + 0.021*"card" + 0.021*"cds" + 0.020*"would" + 0.015*"used" + 0.015*"

# Performance evaluation

In [40]:
#Different words actually present in review number 2000
final_reviews.loc[500, 'reviews_processed']

'opened checking account ally regular walkin bank started bsing around fees fees ally reasonable fair customer service always regardless problem'

In [44]:
#How well the model identifies weightage of each word in a particular review/topic

for index, score in sorted(lda_model[bow_corpus[500]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.35666030645370483	 
Topic: 0.053*"fees" + 0.030*"atm" + 0.024*"fee" + 0.022*"overdraft" + 0.018*"checking" + 0.016*"online" + 0.015*"money" + 0.014*"card" + 0.014*"deposit" + 0.014*"love"

Score: 0.3118644654750824	 
Topic: 0.025*"money" + 0.022*"rates" + 0.021*"accounts" + 0.017*"always" + 0.017*"cd" + 0.017*"would" + 0.016*"one" + 0.015*"time" + 0.015*"opened" + 0.014*"banks"

Score: 0.26782527565956116	 
Topic: 0.037*"savings" + 0.036*"interest" + 0.027*"money" + 0.018*"also" + 0.018*"rates" + 0.017*"online" + 0.017*"fees" + 0.016*"accounts" + 0.016*"rate" + 0.016*"checking"


# Perplexity and Coherence score

In [45]:
texts = final_reviews['reviews_processed'].str.split()
id2word = gensim.corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

#Compute Perplexity -- the lower the better
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))

from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.98108538931216

Coherence Score:  0.45969948526802684


# Visualize the topics-keywords

In [47]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

C:\Users\lenovo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.088930  0.022667       1        1  21.917250
9      0.076554  0.018961       2        1  18.795712
0     -0.154929 -0.000807       3        1  12.873711
6     -0.011133  0.048316       4        1  10.703661
4      0.068562  0.064401       5        1   8.169889
3      0.021133 -0.060969       6        1   7.965651
1      0.043941  0.153461       7        1   6.866108
7     -0.210787 -0.061578       8        1   6.408231
8      0.135397 -0.189271       9        1   3.332178
5     -0.057668  0.004819      10        1   2.967614, topic_info=    Category        Freq      Term       Total  loglift  logprob
65   Default  191.000000        cd  191.000000  30.0000  30.0000
242  Default  244.000000      rate  244.000000  29.0000  29.0000
72   Default  368.000000      fees  368.000000  28.0000  28.0000
206  Default  442.000000  interest  442.000000  27.0000  27.0000
393  Default  227.000000       atm  227.000000  26.0000  26.0000
..       ...         ...       ...         ...      ...      ...
166  Topic10    7.472927      well  113.918694   0.7932  -4.8604
36   Topic10    8.078309     never  202.987518   0.2934  -4.7825
117  Topic10    8.002075     check  216.322235   0.2203  -4.7920
43   Topic10    6.852978      said   81.369827   1.0431  -4.9470
235  Topic10    6.995897   banking  285.123322  -0.1902  -4.9264

[582 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
247       2  0.307866  ability
247       3  0.043981  ability
247       4  0.175923  ability
247       6  0.043981  ability
247       7  0.219904  ability
...     ...       ...      ...
327       1  0.094589     zero
327       3  0.378355     zero
327       5  0.141883     zero
327       9  0.283766     zero
327      10  0.094589     zero

[2109 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 1, 7, 5, 4, 2, 8, 9, 6])

In [49]:
import pickle
Bank_Reviews = 'lda_model.sav'
pickle.dump(lda_model, open(Bank_Reviews, 'wb'))

# SKLearn LDA

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ( "Topic %d:" % (topic_idx) )
        print (" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))


documents = final_reviews['reviews_processed'].apply(str)

no_features = 500

# NMF is able to use tf-idf
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
# tfidf = tfidf_vectorizer.fit_transform(documents)
# tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 10

# Run NMF
# nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)a

# Run LDA
lda = LatentDirichletAllocation(no_topics, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 10
# display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
bank account alli check save money custom servic onlin deposit
Topic 1:
order check receiv payment stop place week speak alli say
Topic 2:
account alli tell close day say time check money ask
Topic 3:
account bank alli check save transfer money order like need
Topic 4:
list acct easili pleas primari add wife differ person email
Topic 5:
bank account alli open servic tell transfer rate matur custom
Topic 6:
follow term figur concern today long save account time recent
Topic 7:
intern joint form info notifi sorri compar exist peopl alli
Topic 8:
card current perk come wast lock ask plan past tradit
Topic 9:
account bank onlin alli save rat custom servic check rate


In [44]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
vis

C:\Users\lenovo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.077796  0.077212       1        1  79.427165
2      0.151889 -0.101418       2        1  11.421862
5      0.138966  0.044706       3        1   7.033814
1     -0.068041 -0.127520       4        1   0.615243
7     -0.048493  0.032930       5        1   0.388123
4     -0.069973  0.023410       6        1   0.263671
6     -0.042948  0.011953       7        1   0.220897
9     -0.036329  0.019295       8        1   0.209742
3     -0.039582  0.007901       9        1   0.209741
8     -0.063285  0.011531      10        1   0.209741, topic_info=    Category         Freq      Term        Total  loglift  logprob
66   Default   736.000000     check   736.000000  30.0000  30.0000
16   Default  1630.000000      alli  1630.000000  29.0000  29.0000
43   Default  1965.000000      bank  1965.000000  28.0000  28.0000
5    Default  1734.000000   account  1734.000000  27.0000  27.0000
390  Default   673.000000      save   673.000000  26.0000  26.0000
..       ...          ...       ...          ...      ...      ...
354  Topic10     0.145733       rat   311.551862  -1.5005  -6.1491
168  Topic10     0.144905      free   115.522915  -0.5141  -6.1548
399  Topic10     0.144613    servic   586.976960  -2.1416  -6.1568
171  Topic10     0.144604  friendli    82.907896  -0.1844  -6.1569
5    Topic10     0.144388   account  1734.397373  -3.2266  -6.1584

[640 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         1  0.850534      abl
1         2  0.074709      abl
1         3  0.068962      abl
2         1  0.607522  absolut
2         2  0.057859  absolut
...     ...       ...      ...
497       3  0.071682     year
497       4  0.003117     year
497       5  0.003117     year
498       1  0.918275    yield
498       3  0.034010    yield

[783 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 6, 2, 8, 5, 7, 10, 4, 9])